In [93]:
import numpy as np
import pandas as pd
import scipy.special as sp
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import PIL
import PIL.Image

import torch
from torch import nn
from torch.autograd import Variable
from torch import optim
from torchvision.transforms import ToTensor

In [101]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Data preparation

In [75]:
import os

df = pd.read_csv('../DL_and_NN_in_Python/fer2013.csv')
df = df.sample(frac=1, random_state=42)

In [ ]:
emotions = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def show_sample_image(emotion: int):
    emotion_no = emotions.index(emotion)
    df_emotion = df[df.emotion == emotion_no]

    random_no = np.random.random_integers(0, len(df_emotion))
    print(random_no)
    print(df_emotion.iloc[random_no])

    img = np.array(list(map(int, df_emotion.iloc[random_no].pixels.split(' '))), dtype=np.uint8).reshape((48,48))
    img = PIL.Image.fromarray(img).resize((1000, 1000))
    img.show()

show_sample_image('Surprise')

2511
emotion                                                    5
pixels     50 28 23 21 18 75 120 102 65 60 64 71 93 104 1...
Usage                                               Training
Name: 17088, dtype: object


/tmp/ipykernel_1318095/1637804024.py:7: DeprecationWarning: This function is deprecated. Please call randint(0, 4002 + 1) instead
  random_no = np.random.random_integers(0, len(df_emotion))


In [150]:
train_proportion = 0.8
train_index = int(train_proportion*len(df))

train_df = df.iloc[:train_index]
test_df = df.iloc[train_index:]

In [151]:
N = len(train_df)
D = len(train_df.iloc[0].pixels.split(' '))
D1 = int(np.sqrt(D))

# or just use train_test_split from sklearn.model_selection for the same effect
n_classes = len(set(train_df.emotion))

print(f'N = {N}, D = {D}, n_classes: {n_classes}')

print(f'Number of samples in training set: {len(train_df)}')
print(f'Number of samples in test set: {len(test_df)}')

N = 28709, D = 2304, n_classes: 7
Number of samples in training set: 28709
Number of samples in test set: 7178


Create a custom Dataset object:

In [153]:
class CustomImageDataset(Dataset):
    def __init__(self, data, transform=None, target_transform=None):
        self.data = data
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.data.iloc[idx, 0]
        image = self.data.iloc[idx, 1]
        image = list((map(int, image.split(' '))))
        image = np.array(image).reshape((D1, D1))
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [156]:
batch_size = 64
train_data = CustomImageDataset(train_df, transform=ToTensor())
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_data = CustomImageDataset(test_df, transform=ToTensor())
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# NNs in PyTorch

The logic behind creating a model in PyTorch is pretty much the same as in Tensorflow. There are some syntactical differences, though.

1. The activation function is not specified as part of the layer, but separately.
2. The input and output dimensions need to be supplied, unlike in Keras, where only output is needed. Note that in pure TF, without Keras, we also need to specify the intput dims unless we postpone the build of the model (see https://www.tensorflow.org/guide/intro_to_modules#waiting_to_create_variables)
3. The input has to be explicitly turned into a torch.Tensor (TF can handle that automatically)

In [33]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [171]:
class ModelPyTorch(torch.nn.Module):
    def __init__(self, M: int = 10):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(D1*D1, M),
            nn.ReLU(),
            nn.Linear(M, n_classes)
        )
        self.double() # ensures input has the same dtype as the parameters

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [172]:
class ModelPyTorchV2(nn.Module):
    def __init__(self, M: int = 10, activation_function = nn.ReLU()):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(D1*D1, M)
        self.layer2 = nn.Linear(M, n_classes)
        self.activation_function = activation_function
        self.double()

    def forward(self, x):
        x = self.flatten(x)
        x = self.activation_function(self.layer1(x))
        x = self.layer2(x)
        return x

In [178]:
def constructNNPyTorch(M: int = 10, activation_function = nn.ReLU()):
    model = nn.Sequential()
    model.add_module('flatten', nn.Flatten())
    model.add_module('name1', nn.Linear(D, M))
    model.add_module('name2', activation_function)
    model.add_module('name3', nn.Linear(M, n_classes))
    model.double()

    return model

In [179]:
model_v1 = ModelPyTorch().to(device)
print(model_v1)
model_v2 = ModelPyTorchV2().to(device)
print(model_v2)
model_v3 = constructNNPyTorch(M=10).to(device)
print(model_v3)

ModelPyTorch(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2304, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=7, bias=True)
  )
)
ModelPyTorchV2(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layer1): Linear(in_features=2304, out_features=10, bias=True)
  (layer2): Linear(in_features=10, out_features=7, bias=True)
  (activation_function): ReLU()
)
Sequential(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (name1): Linear(in_features=2304, out_features=10, bias=True)
  (name2): ReLU()
  (name3): Linear(in_features=10, out_features=7, bias=True)
)
